# Setup

In [1]:
import os
import ctypes as ct

In [2]:
def delink(cppfile,do_print=True):
    """ delinking cppfile is necessary before recompiling 
    (otherwise kernal must be re-started) """
    
    # a. get handle
    handle = cppfile._handle

    # b. delete linking variable
    del cppfile

    # c. free handle
    ct.windll.kernel32.FreeLibrary.argtypes = [ct.wintypes.HMODULE]
    ct.windll.kernel32.FreeLibrary(handle)
    
    if do_print:
        print('cppfile delinked succesfully')
        
def link(filename,do_print=True):
    """ link to filename with hack for getting openmp to work """
    
    # a. load
    cppfile = ct.cdll.LoadLibrary(filename)
    
    # b. setup openmp and delink
    cppfile.setup_omp()
    delink(cppfile,do_print=False)
    
    # c. link again
    cppfile = ct.cdll.LoadLibrary(filename)
    
    if do_print:
        print('cppfile linked succesfully')
    
    return cppfile

# Compile

In [3]:
result = os.system('compile.bat') # build the project in "build/Example.sln"
if result == 0:
    print('cppfile compiled sucessfully')

cppfile compiled sucessfully


# Link

In [4]:
cppfile = link('build/Release/example.dll')

cppfile linked succesfully


# Run

In [5]:
# a. create vectors
diag = np.array([1.5,2.5,3.5])
x = np.zeros(3)

# b. get pointers
p_diag = np.ctypeslib.as_ctypes(diag)
p_x = np.ctypeslib.as_ctypes(x)

# c. call
cppfile.run(p_diag,p_x)

# d. print result
print(x)

[1.50769231 1.26153846 0.64615385]


**Print log.txt:**

In [6]:
with open('log.txt', 'r') as f:
    print(f.read())


%%MatrixMarket matrix coordinate real symmetric
3 3 5
1 1 1.5
2 1 -1
2 2 2.5
3 2 -1
3 3 3.5

x:
0: 1.50769
1: 1.26154
2: 0.646154

omp_get_num_procs() = 8
omp_get_thread_num() = 0
omp_get_thread_num() = 6
omp_get_thread_num() = 2
omp_get_thread_num() = 3
omp_get_thread_num() = 4
omp_get_thread_num() = 5
omp_get_thread_num() = 1
omp_get_thread_num() = 7



# Delink

In [7]:
delink(cppfile)

cppfile delinked succesfully
